<a href="https://colab.research.google.com/github/namdarine/TIF_Chicago_Project/blob/main/Racial_Population.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import library

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from shapely.wkt import loads
import warnings
warnings.filterwarnings("ignore")
import pickle

# Read data

In [3]:
JJ_path = '/content/drive/MyDrive/IPRO497_Demographic Team/Final/data/'
K_path = '/Users/kunwoosmac/Library/Mobile Documents/com~apple~CloudDocs/Python Workspace/IIT Courses/TIF DS Project/Final/data/'
L_path = '/content/drive/MyDrive/IPRO/IPRO 497 TIF Districts/IPRO497_Demographic Team/Final/data/'

path = JJ_path

censusBlock = gpd.read_file(path + 'Boundaries - Census Blocks.geojson')
Community_area = gpd.read_file(path + 'Boundaries - Community Areas (current).geojson')

tifs = pd.read_csv(path + '2019_TIF_Boundaries.csv')
tifs = gpd.GeoDataFrame(tifs)
tifs = tifs.rename(columns={'the_geom': 'geometry'})
tifs['geometry'] = loads(tifs['geometry'])
tifs.set_geometry('geometry', inplace = True)

Blockgroup_boundary = pd.read_csv(path+'cleanBlockGroups.csv')
Blockgroup_boundary = Blockgroup_boundary.drop(columns=['coords'])

Blockgroup_race = pd.read_csv(path + 'Blockgroup_race_2010.csv', header=0)
Blockgroup_race = gpd.GeoDataFrame(Blockgroup_race)


# 1. Preprocessing for demographic
## Skip code -  racial_population = pd.read_csv(path+'racial_population.csv')
### result :


<img src = "https://drive.google.com/uc?id=1xmIG_lcKUMcknxzLBlRWFnikaCpqj0yU">

In [4]:
def preprocess_Race(beforepreprocessed):
  """
  delete unnecessary part of labels

  """

  beforepreprocessed.columns = beforepreprocessed.iloc[0]
  beforepreprocessed = beforepreprocessed.drop(0)
  beforepreprocessed.reset_index(drop=True, inplace=True)
  beforepreprocessed = beforepreprocessed.rename(columns={'Geography': 'geoid'})
  beforepreprocessed = beforepreprocessed.iloc[:, :-1]

  beforepreprocessed.columns = beforepreprocessed.columns.str.replace('!!Total:!!', '')
  beforepreprocessed.columns = beforepreprocessed.columns.str.replace('Total!!', '')

  beforepreprocessed['Geographic Area Name'] = beforepreprocessed["Geographic Area Name"].str.replace(', Cook County, Illinois', '')
  return beforepreprocessed

def get_blockgroup_racial_population(Blockgroup_race_information):
  """
  input : racial population for each census blockgroup
  output:
  columns: 'geoid' 'Geographic Area Name' 'Total Population' 'White Population' 'Black Population' 'Hispanic Population' 'racial_population_list'
  -> total population, white population, black population, hispanic population for each census blockgroup

  """
  hispanic_column = Blockgroup_race_information.columns[3] #All Hispanic combination
  white_column = [Blockgroup_race_information.columns[6]]+ [Blockgroup_race_information.columns[column_name] for column_name in range(14,19)] # All White combinations
  black_column = [Blockgroup_race_information.columns[7]]+[Blockgroup_race_information.columns[column_name] for column_name in range(19,23)] # All Black combinations
  default_column = [Blockgroup_race_information.columns[0],Blockgroup_race_information.columns[1]]  # 'geoid' 'Geographic Area Name'


  # sum
  white_population_info = pd.concat([Blockgroup_race_information[default_column],Blockgroup_race_information[white_column].astype(np.int64).sum(axis=1)],axis=1)
  black_population_info = pd.concat([Blockgroup_race_information[default_column],Blockgroup_race_information[black_column].astype(np.int64).sum(axis=1)],axis=1)
  hispanic_population_info = pd.concat([Blockgroup_race_information[default_column],Blockgroup_race_information[hispanic_column].astype(np.int64)],axis=1)
  total_population_info = Blockgroup_race_information[white_column].astype(np.int64).sum(axis=1)+Blockgroup_race_information[black_column].astype(np.int64).sum(axis=1)+Blockgroup_race_information[hispanic_column].astype(np.int64)
  total_population_info = pd.concat([Blockgroup_race_information[default_column],total_population_info],axis=1)

  # rename
  white_population_info = white_population_info.rename(columns={0:'White Population'})
  black_population_info = black_population_info.rename(columns={0:'Black Population'})
  hispanic_population_info = hispanic_population_info.rename(columns={'Hispanic or Latino':'Hispanic Population'})
  total_population_info = total_population_info.rename(columns={0:'Total Population'})

  # merge
  racial_population_info = pd.merge(total_population_info, white_population_info)
  racial_population_info = pd.merge(racial_population_info, black_population_info)
  racial_population_info = pd.merge(racial_population_info, hispanic_population_info)

  def get_racial_population_list(row):
      return [row['Total Population'], row['White Population'], row['Black Population'], row['Hispanic Population']]

  racial_population_info['racial_population_list']=np.nan
  racial_population_info['racial_population_list'] = racial_population_info.apply(lambda row: get_racial_population_list(row), axis=1)

  return racial_population_info


Blockgroup_race=preprocess_Race(Blockgroup_race)

# Blockgroup_race_2020=preprocess_Race(Blockgroup_race_2020)

racial_population = get_blockgroup_racial_population(Blockgroup_race)

In [5]:
racial_population.head(3)

,geoid,Geographic Area Name,Total Population,White Population,Black Population,Hispanic Population,racial_population_list
0,1500000US170310101001,"Block Group 1, Census Tract 101",550,257,172,121,"[550, 257, 172, 121]"
1,1500000US170310101002,"Block Group 2, Census Tract 101",2247,377,1622,248,"[2247, 377, 1622, 248]"
2,1500000US170310101003,"Block Group 3, Census Tract 101",1850,958,645,247,"[1850, 958, 645, 247]"
